In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import CubicSpline
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel

In [ ]:
from fcmeans import FCM

In [ ]:
from scipy.stats import moment
from scipy.stats import skew

df = pd.read_csv('volume-average-rfile.csv')
df['time'] = df['time'] * 0.01

C_liquid = 1.113e-10 * 1.51 * (10*10)/10
C_gas = 1.113e-10 * 1.000494 * (10*10)/10

def convert_to_capacitance(void_fraction, C_l, C_g):
    return C_l - (void_fraction * (C_l - C_g))

df['capacitance'] = df['Volume fraction'].apply(lambda x: convert_to_capacitance(x, C_liquid, C_gas))

def create_intervals(data, interval_size):
    intervals = []
    stats = []

    for i in range(0, len(data) - interval_size + 1, interval_size):
        interval = data.iloc[i:i+interval_size]

        mean = interval['capacitance'].mean()
        variance = interval['capacitance'].var()
        skewness = skew(interval['capacitance'])
        kurtosis = moment(interval['capacitance'], moment=4)

        stats.append([mean, variance, kurtosis])

    return np.array(stats)

def interpolate_cubic_spline(df, n_points):
    cs = CubicSpline(df['time'], df['Volume fraction'])
    time_new = np.linspace(df['time'].min(), df['time'].max(), n_points)
    void_fraction_new = cs(time_new)
    return pd.DataFrame({'time': time_new, 'Volume fraction': void_fraction_new})

def interpolate_gpr(df, n_points):
    kernel = RBF(length_scale=1.0) + WhiteKernel(noise_level=0.1)
    gpr = GaussianProcessRegressor(kernel=kernel, random_state=0)

    X = df['time'].values.reshape(-1, 1)
    y = df['Volume fraction'].values

    gpr.fit(X, y)

    time_new = np.linspace(df['time'].min(), df['time'].max(), n_points).reshape(-1, 1)
    void_fraction_new, _ = gpr.predict(time_new, return_std=True)

    return pd.DataFrame({'time': time_new.flatten(), 'Volume fraction': void_fraction_new})

n_points = len(df) * 2  # n times more points than original data

df_interpolated = interpolate_cubic_spline(df, n_points)

df_interpolated['capacitance'] = df_interpolated['Volume fraction'].apply(
    lambda x: convert_to_capacitance(x, C_liquid, C_gas)
)

features = create_intervals(df_interpolated, 10)

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

fcm = FCM(n_clusters=3)
fcm.fit(features_scaled)

labels = fcm.u.argmax(axis=1)

fig1 = plt.figure(figsize=(10, 8))
ax1 = fig1.add_subplot(111, projection='3d')
for i in range(3):
    cluster_points = features_scaled[labels == i]
    ax1.scatter(cluster_points[:, 0],
              cluster_points[:, 1],
              cluster_points[:, 2],
              marker=markers[i],
              facecolors='none',
              edgecolors=colors[i],
              s=100,
              label=labels_str[i])

ax1.set_xlabel('Normalized Mean Capacitance')
ax1.set_ylabel('Normalized Variance')
ax1.set_zlabel('Normalized Kurtosis')
ax1.legend()
ax1.set_title('3D View of Flow Patterns')
plt.tight_layout()
fig1.savefig('3d_plot.png', bbox_inches='tight', pad_inches=0.5, dpi=300)

# Create X-Y projection (Mean vs Variance)
fig2 = plt.figure(figsize=(10, 8))
ax2 = fig2.add_subplot(111)
for i in range(3):
    cluster_points = features_scaled[labels == i]
    ax2.scatter(cluster_points[:, 0],
               cluster_points[:, 1],
               marker=markers[i],
               facecolors='none',
               edgecolors=colors[i],
               s=100,
               label=labels_str[i])

ax2.set_xlabel('Normalized Mean Capacitance')
ax2.set_ylabel('Normalized Variance')
ax2.legend()
ax2.set_title('Mean vs Variance View of Flow Patterns')
plt.tight_layout()
fig2.savefig('mean_variance_plot.png', bbox_inches='tight', pad_inches=0.5, dpi=300)

# Create Y-Z projection (Variance vs F95)
fig3 = plt.figure(figsize=(10, 8))
ax3 = fig3.add_subplot(111)
for i in range(3):
    cluster_points = features_scaled[labels == i]
    ax3.scatter(cluster_points[:, 1],
               cluster_points[:, 2],
               marker=markers[i],
               facecolors='none',
               edgecolors=colors[i],
               s=100,
               label=labels_str[i])

ax3.set_xlabel('Normalized Variance')
ax3.set_ylabel('Normalized Kurtosis')
ax3.legend()
ax3.set_title('Variance vs Kurtosis View of Flow Patterns')
plt.tight_layout()
fig3.savefig('variance_kurtosis_plot.png', bbox_inches='tight', pad_inches=0.5, dpi=300)

plt.show()